# Sprint 4: Transformer Fine-tuning (RoBERTa)

**Objective**: Fine-tune RoBERTa on FakeNewsNet titles to leverage deep semantic understanding.

**Hypothesis**: Pre-trained transformers, which understand contextual meaning and nuance, should outperform classical methods (TF-IDF + behavioral features) that rely on bag-of-words representations.

**Baseline to Beat**: 0.8621 ROC-AUC (Hybrid model: TF-IDF + linguistic features)

---

## Setup

In [ ]:
import pandas as pd
import numpy as np
import torch
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score,
    roc_curve, accuracy_score, f1_score
)

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

REPO_DIR = Path('../../')
DATA_PATH = REPO_DIR / 'data' / 'processed'
MODEL_DIR = REPO_DIR / 'models'
RESULTS_DIR = REPO_DIR / 'results'

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {DEVICE}')

## 1. Load Fine-tuned Model & Test Set

In [ ]:
# Load articles and features
df = pd.read_parquet(DATA_PATH / 'articles.parquet')
features = pd.read_parquet(DATA_PATH / 'features.parquet')

df['label_num'] = (df['label'] == 'fake').astype(int)

# Create same 80/20 split as training
from sklearn.model_selection import train_test_split
_, val_df = train_test_split(
    df, test_size=0.2, stratify=df['label_num'], random_state=42
)

print(f'Validation set: {len(val_df)} articles')
print(f'Class distribution:\n{val_df["label_num"].value_counts()}')

## 2. Load Fine-tuned RoBERTa Model

In [ ]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(str(MODEL_DIR / 'roberta_tokenizer'))
model = AutoModelForSequenceClassification.from_pretrained(
    str(MODEL_DIR / 'roberta_fine_tuned'),
    num_labels=2,
    device_map='auto' if DEVICE == 'cuda' else None
)
model.eval()

print('✅ Model loaded successfully')
print(f'Model size: {model.num_parameters():,} parameters')

## 3. Generate Predictions on Validation Set

In [ ]:
# Batch predict
pipe = pipeline(
    'text-classification',
    model=model,
    tokenizer=tokenizer,
    device=0 if DEVICE == 'cuda' else -1,
    batch_size=32
)

results = pipe(val_df['title'].tolist())

# Extract probabilities
y_pred = np.array([1 if r['label'] == 'LABEL_1' else 0 for r in results])
y_pred_proba = np.array([r['score'] for r in results])

# For LABEL_0 predictions, flip probability
y_pred_proba = np.where(
    [r['label'] == 'LABEL_1' for r in results],
    y_pred_proba,
    1 - y_pred_proba
)

y_true = val_df['label_num'].values

print(f'Generated {len(y_pred)} predictions')

## 4. Performance Evaluation

In [ ]:
# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_pred_proba)
cm = confusion_matrix(y_true, y_pred)

print('\n' + '='*70)
print('ROBERTA TRANSFORMER PERFORMANCE')
print('='*70)
print(f'Accuracy:  {accuracy:.4f}')
print(f'F1 Score:  {f1:.4f}')
print(f'ROC-AUC:   {roc_auc:.4f}')
print(f'\nConfusion Matrix:')
print(cm)
print(f'\nTrue Negatives: {cm[0,0]} | False Positives: {cm[0,1]}')
print(f'False Negatives: {cm[1,0]} | True Positives: {cm[1,1]}')

## 5. Comparison with Classical Models

In [ ]:
# Comparison data
comparison_data = {
    'Model': [
        'Behavioral\nOnly',
        'TF-IDF\nBaseline',
        'Hybrid\n(TF-IDF+Behav)',
        'RoBERTa\nTransformer'
    ],
    'Accuracy': [0.5436, 0.8120, 0.8097, accuracy],
    'F1 Score': [0.3930, 0.6443, 0.6412, f1],
    'ROC-AUC': [0.6054, 0.8590, 0.8621, roc_auc]
}

comparison_df = pd.DataFrame(comparison_data)
print('\n' + '='*70)
print('MODEL COMPARISON ACROSS ALL SPRINTS')
print('='*70)
print(comparison_df.to_string(index=False))

# Calculate improvements
baseline_auc = 0.8590
improvement_pct = ((roc_auc - baseline_auc) / baseline_auc) * 100
print(f'\nImprovement over TF-IDF baseline: {improvement_pct:+.2f}%')
print(f'Absolute improvement: {(roc_auc - baseline_auc):+.4f}')

## 6. ROC Curve Comparison

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

# RoBERTa ROC curve
fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
ax.plot(fpr, tpr, color='#d62728', lw=3, label=f'RoBERTa (AUC={roc_auc:.4f})')

# Baseline ROC curves for reference
ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')

ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate', fontsize=12)
ax.set_ylabel('True Positive Rate', fontsize=12)
ax.set_title('RoBERTa Transformer ROC Curve', fontweight='bold', fontsize=13)
ax.legend(loc='lower right', fontsize=11)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'roberta_roc_curve.png', dpi=300, bbox_inches='tight')
plt.show()

print('✅ Saved ROC curve visualization')

## 7. Error Analysis - Where Does RoBERTa Fail?

In [ ]:
# Identify errors
val_df_with_pred = val_df.copy()
val_df_with_pred['y_pred'] = y_pred
val_df_with_pred['y_pred_proba'] = y_pred_proba
val_df_with_pred['correct'] = (val_df_with_pred['label_num'] == y_pred)

false_positives = val_df_with_pred[
    (val_df_with_pred['label_num'] == 0) & (val_df_with_pred['y_pred'] == 1)
]
false_negatives = val_df_with_pred[
    (val_df_with_pred['label_num'] == 1) & (val_df_with_pred['y_pred'] == 0)
]

print(f'\nFalse Positives (Real → Predicted Fake): {len(false_positives)}')
print(f'False Negatives (Fake → Predicted Real): {len(false_negatives)}')
print(f'\nTop 5 High-Confidence False Positives (real but predicted fake):')
print(false_positives.nlargest(5, 'y_pred_proba')[['title', 'label', 'y_pred_proba']].to_string())
print(f'\nTop 5 High-Confidence False Negatives (fake but predicted real):')
print(false_negatives.nsmallest(5, 'y_pred_proba')[['title', 'label', 'y_pred_proba']].to_string())

## 8. Confidence Distribution

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Distribution by true label
ax1.hist([y_pred_proba[y_true == 0], y_pred_proba[y_true == 1]],
         bins=30, label=['Real News', 'Fake News'], alpha=0.7, color=['blue', 'red'])
ax1.set_xlabel('Predicted Probability of Being Fake', fontsize=11)
ax1.set_ylabel('Frequency', fontsize=11)
ax1.set_title('RoBERTa Confidence Distribution', fontweight='bold')
ax1.legend()
ax1.grid(alpha=0.3)

# Distribution by prediction correctness
correct = val_df_with_pred['correct'].values
ax2.hist([y_pred_proba[correct], y_pred_proba[~correct]],
         bins=30, label=['Correct', 'Incorrect'], alpha=0.7, color=['green', 'orange'])
ax2.set_xlabel('Predicted Probability', fontsize=11)
ax2.set_ylabel('Frequency', fontsize=11)
ax2.set_title('Confidence: Correct vs Incorrect Predictions', fontweight='bold')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(RESULTS_DIR / 'roberta_confidence_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print('✅ Saved confidence distribution')

## 9. Key Insights & Next Steps

In [ ]:
print('\n' + '='*70)
print('SPRINT 4 KEY FINDINGS')
print('='*70)

if roc_auc > 0.8621:
    print(f'\n✅ TRANSFORMER BREAKTHROUGH!')
    print(f'   RoBERTa ROC-AUC: {roc_auc:.4f}')
    print(f'   Improvement: {improvement_pct:+.2f}% over hybrid model')
    print(f'\n   Semantic understanding (BERT/RoBERTa) outperformed')
    print(f'   classical bag-of-words approaches.')
else:
    print(f'\n⚠️  MARGINAL IMPROVEMENT')
    print(f'   RoBERTa ROC-AUC: {roc_auc:.4f}')
    print(f'   Improvement: {improvement_pct:+.2f}% over hybrid model')
    print(f'\n   Semantic understanding provides only marginal gains.')
    print(f'   Suggests title-based classification has inherent ceiling.')

print(f'\nFALSE POSITIVES: {len(false_positives)} real articles flagged as fake')
print(f'FALSE NEGATIVES: {len(false_negatives)} fake articles flagged as real')

print(f'\nNEXT STEPS:')
print(f'  1. Analyze error patterns: common topics in FP/FN')
print(f'  2. Attention visualization: which words drive predictions?')
print(f'  3. Domain adaptation: fine-tune on full article text (if available)')
print(f'  4. Ensemble: combine RoBERTa + classical models')
print(f'  5. Production pipeline: model serving + API integration')